# **IMPORTS**

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.utils.data as data
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from tqdm import tqdm
from numpy import *
from numpy.linalg import *
from scipy.special import factorial
from functools import reduce
import random
from torchvision import transforms
import matplotlib.pyplot as plt
import time
import gzip
import cv2
import math
import os
from PIL import Image
from skimage.metrics import structural_similarity as compare_ssim
from skimage.metrics import structural_similarity as ssim
from skimage.transform import resize
import argparse
!pip install lpips
import codecs
import lpips
#!pip install pynvml
#import pynvml

import torch
from torch import nn
from torch import  optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import torchvision as tv
import torch.nn.functional as F
from torchvision.utils import save_image

from PIL import Image
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

from skimage.metrics import structural_similarity as ssim
from skimage.transform import resize
import argparse
!pip install lpips
import codecs
import lpips

import torch
import torch.optim as optim
import torch.nn as nn
import argparse
import os
import random
from torch.autograd import Variable
from torch.utils.data import DataLoader
import numpy as np
import time
from tqdm import trange

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
#from utils_3d import split, merge, psi
import torch.backends.cudnn as cudnn
import functools
from torch.nn import init
import random

# **UTILITY**

In [ ]:
def reshape_patch(img_tensor, patch_size):
    assert 4 == img_tensor.ndim
    seq_length = np.shape(img_tensor)[0]
    img_height = np.shape(img_tensor)[1]
    img_width = np.shape(img_tensor)[2]
    num_channels = np.shape(img_tensor)[3]
    a = np.reshape(img_tensor, [seq_length,
                                img_height // patch_size, patch_size,
                                img_width // patch_size, patch_size,
                                num_channels])
    b = np.transpose(a, [0, 1, 3, 2, 4, 5])
    patch_tensor = np.reshape(b, [seq_length,
                                  img_height // patch_size,
                                  img_width // patch_size,
                                  patch_size * patch_size * num_channels])
    return patch_tensor


def reshape_patch_back(patch_tensor, patch_size):
    # B L H W C
    assert 5 == patch_tensor.ndim
    # patch_tensor = patch_tensor.transpose(0, 1, 3, 4, 5, 2)
    batch_size = np.shape(patch_tensor)[0]
    seq_length = np.shape(patch_tensor)[1]
    patch_height = np.shape(patch_tensor)[2]
    patch_width = np.shape(patch_tensor)[3]
    channels = np.shape(patch_tensor)[4]
    img_channels = channels // (patch_size * patch_size)
    a = np.reshape(patch_tensor, [batch_size, seq_length,
                                  patch_height, patch_width,
                                  patch_size, patch_size,
                                  img_channels])
    b = np.transpose(a, [0, 1, 2, 4, 3, 5, 6])
    img_tensor = np.reshape(b, [batch_size, seq_length,
                                patch_height * patch_size,
                                patch_width * patch_size,
                                img_channels])
    return img_tensor


def reshape_patch_back_tensor(patch_tensor, patch_size):
    # B L H W C
    assert 5 == patch_tensor.ndim
    # patch_tensor = patch_tensor.transpose(0, 1, 3, 4, 5, 2)
    patch_narray = patch_tensor.detach().cpu().numpy()
    batch_size = np.shape(patch_narray)[0]
    seq_length = np.shape(patch_narray)[1]
    patch_height = np.shape(patch_narray)[2]
    patch_width = np.shape(patch_narray)[3]
    channels = np.shape(patch_narray)[4]
    img_channels = channels // (patch_size * patch_size)
    a = torch.reshape(patch_tensor, [batch_size, seq_length,
                                     patch_height, patch_width,
                                     patch_size, patch_size,
                                     img_channels])
    b = a.permute([0, 1, 2, 4, 3, 5, 6])
    img_tensor = torch.reshape(b, [batch_size, seq_length,
                                   patch_height * patch_size,
                                   patch_width * patch_size,
                                   img_channels])
    return img_tensor.permute(0, 1, 4, 2, 3)


def reshape_patch_tensor(img_tensor, patch_size):
    assert 4 == img_tensor.ndim
    seq_length = img_tensor.shape[0]
    img_height = img_tensor.shape[1]
    img_width = img_tensor.shape[2]
    num_channels = img_tensor.shape[3]
    a = torch.reshape(img_tensor, [seq_length,
                                   img_height // patch_size, patch_size,
                                   img_width // patch_size, patch_size,
                                   num_channels])
    b = a.permute((0, 1, 3, 2, 4, 5))
    patch_tensor = torch.reshape(b, [seq_length,
                                     img_height // patch_size,
                                     img_width // patch_size,
                                     patch_size * patch_size * num_channels])
    return patch_tensor.permute((0, 3, 1, 2))

In [ ]:
class Norm(object):
    def __init__(self, max=255):
        self.max = max

    def __call__(self, sample):
        video_x = sample
        new_video_x = video_x / self.max
        return new_video_x


class ToTensor(object):

    def __call__(self, sample):
        video_x = sample
        video_x = video_x.transpose((0, 3, 1, 2))
        video_x = np.array(video_x)
        return torch.from_numpy(video_x).float()
    

class Resize(object):

    def __call__(self, sample):
        imgs_out = np.zeros((
            sample.shape[0], configs.img_height, configs.img_width, sample.shape[3]))
        for i in range(sample.shape[0]):
            imgs_out[i,:,:,:] = resize(sample[i,:,:,:], imgs_out.shape[1:])
        return imgs_out
    
class Resize2(object):

    def __call__(self, sample):
        imgs_out = np.zeros((
            sample.shape[0], 640, 480, sample.shape[3]))
        for i in range(sample.shape[0]):
            imgs_out[i,:,:,:] = resize(sample[i,:,:,:], imgs_out.shape[1:])
        return imgs_out

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def reserve_schedule_sampling_exp(itr):
    if itr < args.r_sampling_step_1:
        r_eta = 0.5
    elif itr < args.r_sampling_step_2:
        r_eta = 1.0 - 0.5 * math.exp(-float(itr - args.r_sampling_step_1) / args.r_exp_alpha)
    else:
        r_eta = 1.0

    if itr < args.r_sampling_step_1:
        eta = 0.5
    elif itr < args.r_sampling_step_2:
        eta = 0.5 - (0.5 / (args.r_sampling_step_2 - args.r_sampling_step_1)) * (itr - args.r_sampling_step_1)
    else:
        eta = 0.0

    r_random_flip = np.random.random_sample(
        (args.batch_size, args.input_length - 1))
    r_true_token = (r_random_flip < r_eta)

    random_flip = np.random.random_sample(
        (args.batch_size, args.total_length - args.input_length - 1))
    true_token = (random_flip < eta)

    ones = np.ones((args.img_height // args.patch_size,
                    args.img_width // args.patch_size,
                    args.patch_size ** 2 * args.img_channel))
    zeros = np.zeros((args.img_height // args.patch_size,
                      args.img_width // args.patch_size,
                      args.patch_size ** 2 * args.img_channel))

    real_input_flag = []
    for i in range(args.batch_size):
        for j in range(args.total_length - 2):
            if j < args.input_length - 1:
                if r_true_token[i, j]:
                    real_input_flag.append(ones)
                else:
                    real_input_flag.append(zeros)
            else:
                if true_token[i, j - (args.input_length - 1)]:
                    real_input_flag.append(ones)
                else:
                    real_input_flag.append(zeros)

    real_input_flag = np.array(real_input_flag)
    real_input_flag = np.reshape(real_input_flag,
                                 (args.batch_size,
                                  args.total_length - 2,
                                  args.img_height // args.patch_size,
                                  args.img_width // args.patch_size,
                                  args.patch_size ** 2 * args.img_channel))
    return real_input_flag


def schedule_sampling(eta, itr):
    zeros = np.zeros((args.batch_size,
                      args.total_length - args.input_length - 1,
                      args.img_height // args.patch_size,
                      args.img_width // args.patch_size,
                      args.patch_size ** 2 * args.img_channel))
    if not args.scheduled_sampling:
        return 0.0, zeros

    if itr < args.sampling_stop_iter:
        eta -= args.sampling_changing_rate
    else:
        eta = 0.0
    random_flip = np.random.random_sample(
        (args.batch_size, args.total_length - args.input_length - 1))
    true_token = (random_flip < eta)
    ones = np.ones((args.img_height // args.patch_size,
                    args.img_width // args.patch_size,
                    args.patch_size ** 2 * args.img_channel))
    zeros = np.zeros((args.img_height // args.patch_size,
                      args.img_width // args.patch_size,
                      args.patch_size ** 2 * args.img_channel))
    real_input_flag = []
    for i in range(args.batch_size):
        for j in range(args.total_length - args.input_length - 1):
            if true_token[i, j]:
                real_input_flag.append(ones)
            else:
                real_input_flag.append(zeros)
    real_input_flag = np.array(real_input_flag)
    real_input_flag = np.reshape(real_input_flag,
                                 (args.batch_size,
                                  args.total_length - args.input_length - 1,
                                  args.img_height // args.patch_size,
                                  args.img_width // args.patch_size,
                                  args.patch_size ** 2 * args.img_channel))
    return eta, real_input_flag

# **DATA LOADER**

In [ ]:
class TimeSeriesDatasetNPZ(data.Dataset):
    def __init__(self, root_dir, n_frames_input=10, n_frames_output=10):
        self.n_frames_in = n_frames_input
        self.n_frames_out = n_frames_output
        random.seed(420)
        n_frames = n_frames_input + n_frames_output
        
        self.file = np.load(root_dir).transpose(1,0,2,3)[..., np.newaxis].transpose(0,1,4,2,3)#[:10]        
            
            
    def __len__(self):
        return len(self.file)

    def __getitem__(self, index):
        clips = torch.from_numpy(self.file[index])
        clips = clips.type(torch.float32)
        clips = (clips / 255)
        return clips

In [ ]:
# td = TimeSeriesDataset(root_dir='../input/casiagdb/GaitDatasetB-silh', n_frames_input=10, n_frames_output=10)
# train_loader = torch.utils.data.DataLoader(dataset=td, batch_size=1, shuffle=True, num_workers=2)

In [ ]:
# z = next(iter(train_loader))
# print(z.shape)
# torch.max(z), torch.min(z)

In [ ]:
class TimeSeriesDatasetNpz(Dataset):
    def __init__(self, root_dir, n_frames_input=10, n_frames_output=10):
        self.n_frames_in = n_frames_input
        self.n_frames_out = n_frames_output
        n_frames = n_frames_input + n_frames_output
        
        self.file = np.load(root_dir).transpose(1,0,2,3)[..., np.newaxis].transpose(0,1,4,2,3)[:5000]          
            
    def __len__(self):
        return len(self.file)

    def __getitem__(self, index):
        clips = torch.from_numpy(self.file[index])
        clips = clips.type(torch.float32)
        clips = (clips / 255)
        return clips # (clips - 0.5) / 0.5  # Norm (-1, 1)

In [ ]:
class TimeSeriesDataset(data.Dataset):
    def __init__(self, root_dir, n_frames_input=10, n_frames_output=10):
        view_type='090'
        random.seed(1000)
        self.n_frames_in = n_frames_input
        self.n_frames_out = n_frames_output
        n_frames = n_frames_input + n_frames_output
        subject_dirs = [d for d in os.listdir(root_dir)] # [01, 02, 03]
        subject_dirs = [os.path.join(root_dir, subject_dirs[i]) for i in range(len(subject_dirs))] # root/01/01/
        seq_type_dirs = [os.path.join(d, sd) for d in subject_dirs for sd in os.listdir(d)] # [root/01/01/bg1, root/01/01/bg2]
        view_type_dirs = sorted([[os.path.join(d, sd)] for d in seq_type_dirs for sd in os.listdir(d) if sd == view_type])

        self.specific_view_files = []
        print(len(view_type_dirs))
        
        for d in view_type_dirs:
            self.specific_view_files.append(sorted([os.path.join(d[0], f) for f in os.listdir(d[0])]))
        
        self.nframes_list = []
        for f in self.specific_view_files[:50]:
            for i in range(len(f)-n_frames):
                self.nframes_list.append(f[i:i+n_frames])
        
        self.trans_norm = transforms.Compose([Norm()])
        self.trans_tensor = transforms.Compose([ToTensor()])
        self.trans_resize = transforms.Compose([Resize()])
        
        print(len(self.nframes_list))
            
            
    def __len__(self):
        return len(self.nframes_list)

    def __getitem__(self, index):
        data_seq = np.ndarray(shape=(len(self.nframes_list[index]), 64, 64), dtype=np.uint8)
        
        for i, f in enumerate(self.nframes_list[index]):
            data_seq[i, :] = cv2.resize(np.array(Image.open(f)), (64, 64))
            
        data_seq = data_seq[..., np.newaxis]
        input = self.trans_norm(data_seq)  
        #input = self.trans_resize(input)
        #input = reshape_patch(input, configs.patch_size)
        input = self.trans_tensor(input)
        return input

## TEST CASE

In [ ]:
# td = TimeSeriesDatasetNpz(root_dir='../input/moving-mnist/mnist_test_seq.npy', n_frames_input=10, n_frames_output=10)
# train_loader = torch.utils.data.DataLoader(dataset=td, batch_size=3, shuffle=True, num_workers=1)

In [ ]:
# z = next(iter(train_loader))
# print(z.shape)
# print(torch.max(z), torch.min(z))

In [ ]:
# plt.figure()
# plt.imshow(z[0][0].permute(1, 2, 0)*0.5 + 0.5, cmap='gray') 
# plt.show()

# **MODELS**

## STCell

In [ ]:
class SpatioTemporalLSTMCell(nn.Module):
    def __init__(self, in_channel, num_hidden, width, filter_size, stride, layer_norm):
        super(SpatioTemporalLSTMCell, self).__init__()

        self.num_hidden = num_hidden
        self.padding = filter_size // 2
        self._forget_bias = 1.0
        if layer_norm:
            self.conv_x = nn.Sequential(
                nn.Conv2d(in_channel, num_hidden * 7, kernel_size=filter_size, stride=stride, padding=self.padding, bias=False),
                nn.LayerNorm([num_hidden * 7, width, width])
            )
            self.conv_h = nn.Sequential(
                nn.Conv2d(num_hidden, num_hidden * 4, kernel_size=filter_size, stride=stride, padding=self.padding, bias=False),
                nn.LayerNorm([num_hidden * 4, width, width])
            )
            self.conv_m = nn.Sequential(
                nn.Conv2d(num_hidden, num_hidden * 3, kernel_size=filter_size, stride=stride, padding=self.padding, bias=False),
                nn.LayerNorm([num_hidden * 3, width, width])
            )
            self.conv_o = nn.Sequential(
                nn.Conv2d(num_hidden * 2, num_hidden, kernel_size=filter_size, stride=stride, padding=self.padding, bias=False),
                nn.LayerNorm([num_hidden, width, width])
            )
        else:
            self.conv_x = nn.Sequential(
                nn.Conv2d(in_channel, num_hidden * 7, kernel_size=filter_size, stride=stride, padding=self.padding, bias=False),
            )
            self.conv_h = nn.Sequential(
                nn.Conv2d(num_hidden, num_hidden * 4, kernel_size=filter_size, stride=stride, padding=self.padding, bias=False),
            )
            self.conv_m = nn.Sequential(
                nn.Conv2d(num_hidden, num_hidden * 3, kernel_size=filter_size, stride=stride, padding=self.padding, bias=False),
            )
            self.conv_o = nn.Sequential(
                nn.Conv2d(num_hidden * 2, num_hidden, kernel_size=filter_size, stride=stride, padding=self.padding, bias=False),
            )
        self.conv_last = nn.Conv2d(num_hidden * 2, num_hidden, kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, x_t, h_t, c_t, m_t):
        x_concat = self.conv_x(x_t)
        h_concat = self.conv_h(h_t)
        m_concat = self.conv_m(m_t)
        i_x, f_x, g_x, i_x_prime, f_x_prime, g_x_prime, o_x = torch.split(x_concat, self.num_hidden, dim=1)
        i_h, f_h, g_h, o_h = torch.split(h_concat, self.num_hidden, dim=1)
        i_m, f_m, g_m = torch.split(m_concat, self.num_hidden, dim=1)

        i_t = torch.sigmoid(i_x + i_h)
        f_t = torch.sigmoid(f_x + f_h + self._forget_bias)
        g_t = torch.tanh(g_x + g_h)

        c_new = f_t * c_t + i_t * g_t

        i_t_prime = torch.sigmoid(i_x_prime + i_m)
        f_t_prime = torch.sigmoid(f_x_prime + f_m + self._forget_bias)
        g_t_prime = torch.tanh(g_x_prime + g_m)

        m_new = f_t_prime * m_t + i_t_prime * g_t_prime

        mem = torch.cat((c_new, m_new), 1)
        o_t = torch.sigmoid(o_x + o_h + self.conv_o(mem))
        h_new = o_t * torch.tanh(self.conv_last(mem))

        return h_new, c_new, m_new


## MAU

In [ ]:
class RNNZZ(nn.Module):
    def __init__(self, num_layers, num_hidden, configs):
        super(RNN, self).__init__()
        # print(configs.srcnn_tf)
        self.configs = configs
        self.frame_channel = configs.patch_size * configs.patch_size * configs.img_channel
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.tau = configs.tau
        self.cell_mode = configs.cell_mode
        self.states = ['recall', 'normal']
        if not self.configs.model_mode in self.states:
            raise AssertionError
        # self.time = 2
        cell_list = []

        width = configs.img_width // configs.patch_size // configs.sr_size
        height = configs.img_height // configs.patch_size // configs.sr_size
        # print(width)

        for i in range(num_layers):
            in_channel = num_hidden[i - 1]
            cell_list.append(
                MAUCell(in_channel, num_hidden[i], height, width, configs.filter_size,
                        configs.stride, self.tau, self.cell_mode)
            )
        self.cell_list = nn.ModuleList(cell_list)

        # Encoder
        n = int(math.log2(configs.sr_size))
        encoders = []
        encoder = nn.Sequential()
        encoder.add_module(name='encoder_t_conv{0}'.format(-1),
                           module=nn.Conv2d(in_channels=self.frame_channel,
                                            out_channels=self.num_hidden[0],
                                            stride=1,
                                            padding=0,
                                            kernel_size=1))
        encoder.add_module(name='relu_t_{0}'.format(-1),
                           module=nn.LeakyReLU(0.2))
        encoders.append(encoder)
        for i in range(n):
            encoder = nn.Sequential()
            encoder.add_module(name='encoder_t{0}'.format(i),
                               module=nn.Conv2d(in_channels=self.num_hidden[0],
                                                out_channels=self.num_hidden[0],
                                                stride=(2, 2),
                                                padding=(1, 1),
                                                kernel_size=(3, 3)
                                                ))
            # self.encoder_t.add_module(name='gn_t{0}'.format(i),
            #                           module=nn.GroupNorm(4, self.frame_channel))
            encoder.add_module(name='encoder_t_relu{0}'.format(i),
                               module=nn.LeakyReLU(0.2))
            encoders.append(encoder)
        self.encoders = nn.ModuleList(encoders)

        # Decoder
        decoders = []

        for i in range(n - 1):
            decoder = nn.Sequential()
            decoder.add_module(name='c_decoder{0}'.format(i),
                               module=nn.ConvTranspose2d(in_channels=self.num_hidden[-1],
                                                         out_channels=self.num_hidden[-1],
                                                         stride=(2, 2),
                                                         padding=(1, 1),
                                                         kernel_size=(3, 3),
                                                         output_padding=(1, 1)
                                                         ))
            # self.decoder_s.add_module(name='gn_decoder_s{0}'.format(i),
            #                           module=nn.GroupNorm(4, self.frame_channel))
            decoder.add_module(name='c_decoder_relu{0}'.format(i),
                               module=nn.LeakyReLU(0.2))
            decoders.append(decoder)

        if n > 0:
            decoder = nn.Sequential()
            decoder.add_module(name='c_decoder{0}'.format(n - 1),
                               module=nn.ConvTranspose2d(in_channels=self.num_hidden[-1],
                                                         out_channels=self.num_hidden[-1],
                                                         stride=(2, 2),
                                                         padding=(1, 1),
                                                         kernel_size=(3, 3),
                                                         output_padding=(1, 1)
                                                         ))
            decoders.append(decoder)
        self.decoders = nn.ModuleList(decoders)

        self.srcnn = nn.Sequential(
            nn.Conv2d(self.num_hidden[-1], self.frame_channel, kernel_size=1, stride=1, padding=0)
        )
        self.merge = nn.Conv2d(self.num_hidden[-1] * 2, self.num_hidden[-1], kernel_size=1, stride=1, padding=0)
        self.conv_last_sr = nn.Conv2d(self.frame_channel * 2, self.frame_channel, kernel_size=1, stride=1, padding=0)

    def forward(self, frames, mask_true, verbose=False):
        # print('ok')
        mask_true = mask_true.permute(0, 1, 4, 2, 3).contiguous()
        if(verbose):
            print('MT Permuted to: ',  mask_true.shape)
        batch_size = frames.shape[0]
        height = frames.shape[3] // self.configs.sr_size
        width = frames.shape[4] // self.configs.sr_size
        frame_channels = frames.shape[2]
        next_frames = []
        T_t = []
        T_pre = []
        S_pre = []
        # H_t = []
        x_gen = None
        if(verbose):
            print('Num Layers: ', self.num_layers)
            print('Num Hidden: ', self.num_hidden)
            print('TAU: ', self.tau)
        for layer_idx in range(self.num_layers):
            tmp_t = []
            tmp_s = []
            if layer_idx == 0:
                in_channel = self.num_hidden[layer_idx]
            else:
                in_channel = self.num_hidden[layer_idx - 1]
            for i in range(self.tau):
                if(verbose):
                    if i==2:
                        print('tmp_t[1]', tmp_t[1].shape)
                tmp_t.append(torch.zeros([batch_size, in_channel, height, width]).to(self.configs.device))
                tmp_s.append(torch.zeros([batch_size, in_channel, height, width]).to(self.configs.device))
            T_pre.append(tmp_t)
            S_pre.append(tmp_s)

        if(verbose):
            print('len T_pre', len(T_pre))
            print('len T_pre[1]', len(T_pre[1]))

        for t in range(self.configs.total_length - 1):
            if t < self.configs.input_length:
                net = frames[:, t]
                if(verbose):
                    print('Net frames[:, t]', frames[:, t].shape)
            else:
                time_diff = t - self.configs.input_length
                net = mask_true[:, time_diff] * frames[:, t] + (1 - mask_true[:, time_diff]) * x_gen
                if(verbose):
                    print('Net', net.shape)

            frames_feature = net
            frames_feature_encoded = []
            if(verbose):
                print('Len Encoders', len(self.encoders))
            for i in range(len(self.encoders)):
                frames_feature = self.encoders[i](frames_feature)
                if(verbose):
                    print('Frames_feature', i, frames_feature.shape)
                frames_feature_encoded.append(frames_feature)
            if t == 0:
                for i in range(self.num_layers):
                    zeros = torch.zeros([batch_size, self.num_hidden[i], height, width]).to(self.configs.device)
                    T_t.append(zeros)
                    # print('ok')
            S_t = frames_feature
            if(verbose):
                print('S_t in', S_t.shape)
            for i in range(self.num_layers):
                t_att = T_pre[i][-self.tau:]
                t_att = torch.stack(t_att, dim=0)
                s_att = S_pre[i][-self.tau:]
                s_att = torch.stack(s_att, dim=0)
                S_pre[i].append(S_t)
                T_t[i], S_t = self.cell_list[i](T_t[i], S_t, t_att, s_att)
                T_pre[i].append(T_t[i])
                # S_pre[i].append(S_t)
            out = S_t
            if(verbose):
                print('S_t out', S_t.shape)
            # out = self.merge(torch.cat([T_t[-1], S_t], dim=1))
            frames_feature_decoded = []
            for i in range(len(self.decoders)):
                out = self.decoders[i](out)
                if(verbose):
                    print('S_t out', i, out.shape)
                # print("ok")
                if self.configs.model_mode == 'recall':
                    # print('unet')
                    out = out + frames_feature_encoded[-2 - i]
                    if(verbose):
                        print('S_t out unet', i, out.shape)
            # out = self.decoder(out)

            x_gen = self.srcnn(out)
            next_frames.append(x_gen)
            if(verbose):
                print('x_gen', x_gen.shape)
                print('len next_frames', len(next_frames))
        if(verbose):
            print('len next_frames FULL', len(next_frames))
        next_frames = torch.stack(next_frames, dim=0)
        if(verbose):
            print('next_frames Tensor', next_frames.shape)
        next_frames = next_frames.permute(1, 0, 2, 3, 4).contiguous()
        if(verbose):
            print('next_frames Tensor Permuted', next_frames.shape)
        return next_frames

In [ ]:
class RNN(nn.Module):
    def __init__(self, num_layers, num_hidden, configs):
        super(RNN, self).__init__()

        self.configs = configs
        self.frame_channel = configs.patch_size * configs.patch_size * configs.img_channel
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        cell_list = []

        width = configs.img_width // configs.patch_size
        self.MSE_criterion = nn.MSELoss()

        for i in range(num_layers):
            in_channel = self.frame_channel if i == 0 else num_hidden[i - 1]
            cell_list.append(
                SpatioTemporalLSTMCell(in_channel, num_hidden[i], width, configs.filter_size,
                                       configs.stride, configs.layer_norm)
            )
        self.cell_list = nn.ModuleList(cell_list)
        self.conv_last = nn.Conv2d(num_hidden[num_layers - 1], self.frame_channel,
                                   kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, frames_tensor, mask_true):
        # [batch, length, height, width, channel] -> [batch, length, channel, height, width]
        #frames = frames_tensor.permute(0, 1, 4, 2, 3).contiguous()
        frames = frames_tensor
        mask_true = mask_true.permute(0, 1, 4, 2, 3).contiguous()

        batch = frames.shape[0]
        height = frames.shape[3]
        width = frames.shape[4]

        next_frames = []
        h_t = []
        c_t = []

        for i in range(self.num_layers):
            zeros = torch.zeros([batch, self.num_hidden[i], height, width]).to(self.configs.device)
            h_t.append(zeros)
            c_t.append(zeros)

        memory = torch.zeros([batch, self.num_hidden[0], height, width]).to(self.configs.device)

        for t in range(self.configs.total_length - 1):
            # reverse schedule sampling
            if self.configs.reverse_scheduled_sampling == 1:
                if t == 0:
                    net = frames[:, t]
                else:
                    net = mask_true[:, t - 1] * frames[:, t] + (1 - mask_true[:, t - 1]) * x_gen
            else:
                if t < self.configs.input_length:
                    net = frames[:, t]
                else:
                    net = mask_true[:, t - self.configs.input_length] * frames[:, t] + \
                          (1 - mask_true[:, t - self.configs.input_length]) * x_gen

            h_t[0], c_t[0], memory = self.cell_list[0](net, h_t[0], c_t[0], memory)

            for i in range(1, self.num_layers):
                h_t[i], c_t[i], memory = self.cell_list[i](h_t[i - 1], h_t[i], c_t[i], memory)

            x_gen = self.conv_last(h_t[self.num_layers - 1])
            next_frames.append(x_gen)

        # [length, batch, channel, height, width] -> [batch, length, height, width, channel]
        next_frames = torch.stack(next_frames, dim=0).permute(1, 0, 2, 3, 4).contiguous()
        loss = self.MSE_criterion(next_frames, frames_tensor[:, 1:])
        return next_frames, loss, loss

# **TRAINER TESTER**

In [ ]:
def train(model, ims, real_input_flag, configs, itr, val):
    _, loss_l1, loss_l2 = model.train(ims, real_input_flag, itr, val)
    if itr % configs.display_interval == 0:
        print('Step: ' + str(itr),
              'Training L1 loss: ' + str(loss_l1), 'Training L2 loss: ' + str(loss_l2))
    return loss_l1, loss_l2

In [ ]:
def evaluation_proper(model, test_loader, configs, out_len=10):
    print('Evaluating...')
    
    loss_fn = lpips.LPIPS(net='alex', spatial=True).to(configs.device)
    mse_list = np.empty((len(test_loader), out_len))
    mae_list = np.empty((len(test_loader), out_len))
    ssim_list = np.empty((len(test_loader), out_len))
    psnr_list = np.empty((len(test_loader), out_len))
    lpips_list = np.empty((len(test_loader), out_len))
    
    total_mse = 0
    total_mae = 0
    
    with torch.no_grad():
        #model.eval()
        for i, data in tqdm(enumerate(test_loader, 0), total=len(test_loader)):
 
            batch_size = data.shape[0]
            real_input_flag = np.zeros(
                (batch_size,
                 configs.total_length - configs.input_length - 1,
                 configs.img_height // configs.patch_size,
                 configs.img_width // configs.patch_size,
                 configs.patch_size ** 2 * configs.img_channel))

            img_gen = model.test(data, real_input_flag)
            img_gen = img_gen.transpose(0, 1, 3, 4, 2)  # * 0.5 + 0.5
            test_ims = data.detach().cpu().numpy().transpose(0, 1, 3, 4, 2)  # * 0.5 + 0.5
            output_length = configs.total_length - configs.input_length
            output_length = min(output_length, configs.total_length - 1)
            test_ims = reshape_patch_back(test_ims, configs.patch_size)
            img_gen = reshape_patch_back(img_gen, configs.patch_size)
            target = data[:, configs.input_length:, :].detach().cpu().numpy().transpose(0, 1, 3, 4, 2)
            predictions = img_gen[:, -output_length:, :]
            
            predictions[predictions < 0.10] = 0
            predictions[predictions > 0.99] = 1
            
            if (i+1) % 500 == 0:
                print(target[0, 1, 40:42, 40:42, 0])
                print(predictions[0, 1, 40:42, 40:42, 0])
                fig, ax = plt.subplots(2, out_len, figsize=(25, 7))
                for i in range(2):
                    for j in range(out_len):
                        if i == 0:
                            ax[i][j].imshow(target[0][j])
                            ax[i][j].set_title('V Ground Truth')
                        if i == 1:
                            ax[i][j].imshow(predictions[0][j])
                            ax[i][j].set_title('V Generated')
                        ax[i][j].axis('off')
                plt.show()
            
            mse_batch = np.mean((predictions-target)**2 , axis=(0,1,4)).sum()
            mae_batch = np.mean(np.abs(predictions-target),  axis=(0,1,4)).sum() 
            total_mse += mse_batch
            total_mae += mae_batch
            
            for j in range(out_len):
                mse_list[i][j] = np.square(predictions[:,j,:,:,:] - target[:,j,:,:,:]).mean()
                mae_list[i][j] = np.abs(predictions[:,j,:,:,:] - target[:,j,:,:,:]).mean()
                ssim_list[i][j] = ssim(target[0,j,:,:,0], predictions[0,j,:,:,0], multichannel=False)
                psnr_list[i][j] = 20 * np.log10(1 / sqrt(mse_list[i][j]))
                t1 = torch.from_numpy((predictions[:,j,:,:,:] - 0.5) / 0.5).to(configs.device).permute((0, 3, 1, 2))
                t2 = torch.from_numpy((target[:,j,:,:,:] - 0.5) / 0.5).to(configs.device).permute((0, 3, 1, 2))
                d = loss_fn.forward(t1, t2)
                lpips_list[i][j] = d.mean().detach().cpu().numpy() * 100
                    
        #model.train()
        
    avg_mse_frame = mse_list.mean(axis=0)
    avg_mae_frame = mae_list.mean(axis=0)
    avg_ssim_frame = ssim_list.mean(axis=0)
    avg_psnr_frame = psnr_list.mean(axis=0)
    avg_lpips_frame = lpips_list.mean(axis=0)

    avg_mse = mse_list.mean()
    avg_mae = mae_list.mean()
    avg_ssim = ssim_list.mean()
    avg_psnr = psnr_list.mean()
    avg_lpips = lpips_list.mean()

    print('Eval MSE: ', total_mse/(len(test_loader)))
    print('Eval MAE: ', total_mae/(len(test_loader)))
    
    print(f'Avg-MSE: {avg_mse}\nMSE/Frame: {avg_mse_frame}')
    print(f'Avg-MAE: {avg_mae}\nMAE/Frame: {avg_mae_frame}')
    print(f'Avg-SSIM: {avg_ssim}\nSSIM/Frame: {avg_ssim_frame}')
    print(f'Avg-PSNR: {avg_psnr}\nPSNR/Frame: {avg_psnr_frame}')
    print(f'Avg-LPIPS: {avg_lpips}\nLPIPS/Frame: {avg_lpips_frame}')
    
    return avg_mse

# **TRAIN TEST WRAPPER**

In [ ]:
def train_wrapper(model):
    begin = 0
    
    if args.pretrained_model:
        model.load(args.pretrained_model)
        begin = int(args.pretrained_model.split('-')[-1])

        
    # DATASET
    dataset = TimeSeriesDataset(root_dir=configs.data_train_path, n_frames_input=10, n_frames_output=10)
    
    # DATA LOADER + SPLIT
    validation_split = .3
    shuffle_dataset = True
    random_seed= 1000

    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]
    
    model.load('../input/predv1caswts/predv1forcasia.ckpt-4000')

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    train_input_handle = torch.utils.data.DataLoader(dataset, args.batch_size, sampler=train_sampler, num_workers=2, pin_memory=True, drop_last=True)
    val_input_handle = torch.utils.data.DataLoader(dataset, batch_size=4, sampler=valid_sampler, drop_last=True)

    losses_l1 = []
    losses_l2 = []
    
    eta = args.sampling_start_value
    eta -= (begin * args.sampling_changing_rate)
    itr = begin
    # real_input_flag = {}
    for epoch in range(0, args.max_epoches):
        print('E:', epoch)
            
        if epoch == 0:
            evaluation_proper(model, val_input_handle, configs, out_len=10)
        
        for ims in tqdm(train_input_handle, total=len(train_input_handle)):
            if itr > args.max_iterations:
                break
            batch_size = ims.shape[0]
            if(configs.verbose):
                print('IMS shape: ', ims.shape)
                print('Stuff input to schedule sampling: ', eta, itr, args.img_channel, batch_size)
            eta, real_input_flag = schedule_sampling(eta, itr)
            if(configs.verbose):
                print('Stuff output from schedule sampling: ', eta, real_input_flag.shape)

            l1, l2 = train(model, ims, real_input_flag, args, itr, next(iter(val_input_handle)))
            losses_l1.append(l1.item())
            losses_l2.append(l2.item())
            
            if itr % configs.plot_interval == 0:
                fig, ax = plt.subplots(2, 1, figsize=(13, 5))
                a = ax.flatten()
                a[0].plot(losses_l1, 'r')
                a[0].set_title('Loss L1')
                a[1].plot(losses_l2, 'r')
                a[1].set_title('Loss L2')
                plt.show()
            
            if itr % args.snapshot_interval == 0 and itr > begin:
                model.save(itr)
            itr += 1

        if epoch >= 8:
            evaluation_proper(model, val_input_handle, configs, out_len=10)
            
        #evaluation_proper(model, val_input_handle, configs, out_len=10)
            
#             meminfo_end = pynvml.nvmlDeviceGetMemoryInfo(handle)
#             if(configs.verbose):
#                 print("GPU memory:%dM" % ((meminfo_end.used - meminfo_begin.used) / (1024 ** 2)))


def test_wrapper(model, val_ds):
    #model.load(args.pretrained_model)
    test_input_handle = val_ds

#     itr = 1
#     for i in range(itr):
    trainer.test(model, test_input_handle, args, itr)

# **MODEL FACTORY**

In [ ]:
class Model(object):
    def __init__(self, configs):
        self.configs = configs
        self.patch_height = configs.img_height // configs.patch_size
        self.patch_width = configs.img_width // configs.patch_size
        self.patch_channel = configs.img_channel * (configs.patch_size ** 2)
        self.num_layers = configs.num_layers
        networks_map = {
            'predv2': RNN,
        }
        num_hidden = []
        for i in range(configs.num_layers):
            num_hidden.append(configs.num_hidden)
        self.num_hidden = num_hidden
        if configs.model_name in networks_map:
            Network = networks_map[configs.model_name]
            self.network = Network(self.num_layers, self.num_hidden, configs).to(configs.device)
        else:
            raise ValueError('Name of network unknown %s' % configs.model_name)

        self.optimizer = Adam(self.network.parameters(), lr=configs.lr)
        self.MSE_criterion = nn.MSELoss()
        self.L1_loss = nn.L1Loss()

    def save(self, itr):
        stats = {'net_param': self.network.state_dict()}
        checkpoint_path = os.path.join(self.configs.save_dir, 'model.ckpt' + '-' + str(itr))
        torch.save(stats, checkpoint_path)
        print("save predictive model to %s" % checkpoint_path)

    def load(self, pm_checkpoint_path):
        print('load predictive model:', pm_checkpoint_path)
        stats = torch.load(pm_checkpoint_path, map_location=torch.device(self.configs.device))
        self.network.load_state_dict(stats['net_param'])

    def train(self, data, mask, itr, val):
        frames = data
        self.network.train()
        val_tensor = torch.FloatTensor(val).to(self.configs.device)
        frames_tensor = torch.FloatTensor(frames).to(self.configs.device)
        mask_tensor = torch.FloatTensor(mask).to(self.configs.device)

        if(self.configs.verbose):
            print('FT', frames_tensor.shape)
            print('MT', mask_tensor.shape)
        
        next_frames, loss1, loss2 = self.network(frames_tensor, mask_tensor)
        loss = loss1 + loss2
        if(self.configs.verbose):
            print('Next Frames', next_frames.shape)
            
        #next_frames = next_frames.permute(0, 1, 4, 2, 3)

        ground_truth = frames_tensor
        if(self.configs.verbose):
            print('Ground', ground_truth[:, 1:].shape)

            
        
        if itr % configs.plot_interval == 0:
            with torch.no_grad():
                self.network.eval()
                x = frames_tensor[0][0:configs.input_length]
                y = frames_tensor[0][configs.input_length:]
                g = next_frames[0][configs.input_length-1:]
                m = mask_tensor[0]
                fig, ax = plt.subplots(4, configs.input_length, figsize=(25, 10))
                for i in range(4):
                    for j in range(configs.input_length):
                        if i == 0:
                            ax[i][j].imshow(x[j].to('cpu').permute(1, 2, 0))
                            ax[i][j].set_title('T Input')
                        if i == 1:
                            if j == configs.input_length-1:
                                ax[i][j].axis('off')
                                continue
                            ax[i][j].imshow(m[j].to('cpu'))
                            ax[i][j].set_title('T Mask')
                        if i == 2:
                            ax[i][j].imshow(y[j].to('cpu').permute(1, 2, 0))
                            ax[i][j].set_title('T Ground Truth')
                        if i == 3:
                            ax[i][j].imshow(g[j].detach().to('cpu').permute(1, 2, 0))
                            ax[i][j].set_title('T Generated')
                        ax[i][j].axis('off')


#                 x = val_tensor[0][0:configs.input_length]
#                 y = val_tensor[0][configs.input_length:]
#                 mask = torch.zeros_like(mask_tensor[0]).unsqueeze(0).to(configs.device)
#                 next_frameszz = self.network(val_tensor, mask)
#                 m = mask[0]
#                 g = next_frameszz[0][configs.input_length-1:]
#                 fig, ax = plt.subplots(4, configs.input_length, figsize=(25, 10))
#                 for i in range(4):
#                     for j in range(configs.input_length):
#                         if i == 0:
#                             ax[i][j].imshow(x[j].to('cpu').permute(1, 2, 0))
#                             ax[i][j].set_title('V Input')
#                         if i == 1:
#                             if j == configs.input_length-1:
#                                 ax[i][j].axis('off')
#                                 continue
#                             ax[i][j].imshow(m[j].to('cpu'))
#                             ax[i][j].set_title('V Mask')
#                         if i == 2:
#                             ax[i][j].imshow(y[j].to('cpu').permute(1, 2, 0))
#                             ax[i][j].set_title('V Ground Truth')
#                         if i == 3:
#                             ax[i][j].imshow(g[j].detach().to('cpu').permute(1, 2, 0))
#                             ax[i][j].set_title('V Generated')
#                         ax[i][j].axis('off')
                
                self.network.train()
            
                    
            
        batch_size = next_frames.shape[0]

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

#         if itr >= self.configs.sampling_stop_iter and itr % self.configs.delay_interval == 0:
#             self.scheduler.step()
#             # self.scheduler_F.step()
#             # self.scheduler_D.step()
#             print('LR decay to:%.8f', self.optimizer.param_groups[0]['lr'])

        return next_frames, loss1.detach().cpu().numpy(), loss2.detach().cpu().numpy()

    def test(self, data, mask):
        frames = data
        self.network.eval()
        frames_tensor = torch.FloatTensor(frames).to(self.configs.device)
        mask_tensor = torch.FloatTensor(mask).to(self.configs.device)
        next_frames, loss1, loss2 = self.network(frames_tensor, mask_tensor)
        return next_frames.detach().cpu().numpy()

# **CONFIG**

In [ ]:
class Configuration:
    def __init__(self):
        super(Configuration, self).__init__()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.data_train_path = '../input/casiabpretreated/Here'
        self.data_test_path = '../input/kthextract-to-jpg/data/test'
        self.input_length = 10
        self.real_length = 20
        self.total_length = 20
        self.img_height = 64
        self.img_width = 64
        self.sr_size = 4
        self.img_channel = 1
        self.patch_size = 1
        self.alpha = 1
        self.model_name = 'predv2'
        self.dataset = 'mmnist'
        self.cell_mode = 'residual'
        self.model_mode = 'recall'
        self.num_workers = 2
        self.num_hidden = 32
        self.num_layers = 2
        self.num_heads = 2
        self.filter_size = 5
        self.stride = 1
        self.time = 2
        self.time_stride = 1
        self.tau = 7
        self.is_training = True
        self.lr = 1e-3
        self.lr_decay = 0.90
        self.delay_interval = 2000
        self.batch_size = 32
        self.max_iterations = 150000
        self.max_epoches = 10
        self.display_interval = 150
        self.plot_interval = 150
        self.test_interval = 1010
        self.snapshot_interval = 1000
        self.num_save_samples = 3
        self.n_gpu = 1
        self.pretrained_model = ''
        self.perforamnce_dir = 'results/mmnist'
        self.save_dir = 'saves/mmnist'
        self.gen_frm_dir = 'results/mmnist/'
        self.scheduled_sampling = True
        self.sampling_stop_iter = 50000
        self.sampling_start_value = 1.0
        self.sampling_changing_rate = 0.000005
        self.verbose = False
        self.visual = 0
        self.visual_path=''
        self.layer_norm=0
        self.reverse_scheduled_sampling=0
        self.decouple_beta=0.1
        
configs = Configuration()
args = configs

## TEST CASE

In [ ]:
def test_model(configs):
    nl = 4
    nh = [64, 64, 64, 64]
    z = torch.randn(1, 20, 1, 256, 256).to(configs.device)
    m = torch.zeros(1, 9, 256, 256, 1).to(configs.device)
    model = RNN(nl, nh, configs).to(configs.device)
    g = model(z, m, True)
    print(g.shape)

#test_model(configs)

In [ ]:
#model = Model(args)

In [ ]:
#count_parameters(model.network)

# **TRAIN**

In [ ]:
#pynvml.nvmlInit()

print('Initializing models')

model = Model(args)

if args.is_training:
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)
    if not os.path.exists(args.gen_frm_dir):
        os.makedirs(args.gen_frm_dir)
    train_wrapper(model)
else:
    if not os.path.exists(args.gen_frm_dir):
        os.makedirs(args.gen_frm_dir)
    test_wrapper(model)